# Requirements

In [43]:
import Pkg
try
    Pkg.generate("Pipelines")
catch(err)
end
Pkg.activate("Pipelines")
Pkg.instantiate()
Pkg.add("Transducers")
Pkg.add("BenchmarkTools")
using BenchmarkTools
using Transducers

  Activating project at `~/Projects/Julia_good_bad_ugly/source-code/semantics/Pipelines`
   Resolving package versions...
  No Changes to `~/Projects/Julia_good_bad_ugly/source-code/semantics/Pipelines/Project.toml`
  No Changes to `~/Projects/Julia_good_bad_ugly/source-code/semantics/Pipelines/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Projects/Julia_good_bad_ugly/source-code/semantics/Pipelines/Project.toml`
  No Changes to `~/Projects/Julia_good_bad_ugly/source-code/semantics/Pipelines/Manifest.toml`


# Classic pipelines

The usual way of creating a data processing pipeline is by nesting function calls.

For example, suppose you want the sum of all prime numbers upto 100, you would apply a filter to a range, and compute the sum of the result.

In [36]:
function isprime(n::Integer)
    n > 2 && return all(map(i -> n % i != 0, 2:ceil(sqrt(n))))
    return n == 2
end

isprime (generic function with 1 method)

In [37]:
sum(filter(isprime, 1:100))

1060

# Pipe operator

This however requires us to read from the inside out, which is somewhat hard to do.  Using Julia's pipe operator `|>` the code can be made easier to read.

In [38]:
filter(isprime, 1:100) |> sum

1060

However, you can not easily compose functions such as `map`, `filter` and `reduce` in this way, unless you use, e.g., the `Transducers` package.

# Transducers

Consider for instance the task of computing the square of all prime numbers up to 100.  Using the classic approach, that gets pretty tedious.

In [39]:
sum(
    map(i -> i^2,
        filter(isprime,
            1:100
            )
        )
    )

65796

This is much easier to interprete using `Transformers`.

In [40]:
1:100 |> Filter(isprime) |> Map(i -> i^2) |> sum

65796

# Performance & laziness

An important question for such pipelines is whether they are lazy or not, i.e, do they finish as soon as possible?  A good way to check this is to generate a long vector of positive numbers, and replace the first component by a negative number, map to `Bool` and feed the result to `all`.  If this is lazy, it should be obvious.

In [63]:
all_postive = collect(1:100_000_000);

In [64]:
@benchmark all(map(i -> i > 0, map(i -> i^3, all_postive)))

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  165.243 ms … 299.222 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     184.554 ms               ┊ GC (median):    0.84%
 Time  (mean ± σ):   191.224 ms ±  29.029 ms  ┊ GC (mean ± σ):  1.13% ± 1.18%

   ▁ █▁ ▁ ▄▄                      ▁                              
  ▆█▆██▁█▆██▁▁▆▆▆▁▁▁▁▁▁▆▁▁▁▁▁▆▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  165 ms           Histogram: frequency by time          299 ms <

 Memory estimate: 858.31 MiB, allocs estimate: 6.

In [65]:
one_negative = copy(all_postive);
one_negative[1] = -1;

In [66]:
@benchmark all(map(i -> i > 0, map(i -> i^3, one_negative)))

BenchmarkTools.Trial: 26 samples with 1 evaluation.
 Range (min … max):  166.015 ms … 299.337 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     186.737 ms               ┊ GC (median):    1.22%
 Time  (mean ± σ):   194.199 ms ±  33.917 ms  ┊ GC (mean ± σ):  1.19% ± 1.19%

  ▁ ▁ ▄    █▄ ▁ ▁                                                
  █▆█▆█▁▁▆▆██▁█▁█▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▆▁▁▁▁▁▁▁▁▆ ▁
  166 ms           Histogram: frequency by time          299 ms <

 Memory estimate: 858.31 MiB, allocs estimate: 6.

It is clear that the composition of these functions is not lazy, although individual functions might be.

In [67]:
@benchmark all_postive |> Map(i -> i^3) |> Map(i -> i > 0) |> all

BenchmarkTools.Trial: 54 samples with 1 evaluation.
 Range (min … max):  56.797 ms … 430.671 ms  ┊ GC (min … max):  0.00% … 29.04%
 Time  (median):     65.566 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   95.734 ms ±  77.512 ms  ┊ GC (mean ± σ):  14.67% ± 17.46%

   █                                                            
  ▇█▁▁▁▁▅▅▁▁▁▅▁▁▁▁▁▆▅▁▁▁▁▆▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▅ ▁
  56.8 ms       Histogram: log(frequency) by time       363 ms <

 Memory estimate: 96.00 MiB, allocs estimate: 2097157.

In [68]:
@benchmark one_negative |> Map(i -> i^3) |> Map(i -> i > 0) |> all

BenchmarkTools.Trial: 10000 samples with 800 evaluations.
 Range (min … max):  155.115 ns … 159.172 μs  ┊ GC (min … max):  0.00% … 99.82%
 Time  (median):     208.239 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   301.909 ns ±   2.248 μs  ┊ GC (mean ± σ):  10.44% ±  1.41%

  ▅▃█▃▂▂▃                                                  ▁▁▁  ▁
  ████████▇▅▆▅▄▅▁▃▁▁▁▃▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▃▁▁▁▅█████ █
  155 ns        Histogram: log(frequency) by time       1.63 μs <

 Memory estimate: 256 bytes, allocs estimate: 6.

The functions in `Transducers` are faster than their Julia `Base` counterparts, but more importantly, composition is lazy as illustrated by the last two benchmarks.